~ 1.5 hour runtime

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON, SPARQLExceptions
import pandas as pd
from typing import List
from tqdm.notebook import tqdm
import time


In [2]:
result_lst = []

In [ ]:
categories_to_scan = ['wd:Q38723', 'wd:Q189004', 'wd:Q3918', 'wd:Q62078547', 'wd:Q269770', 'wd:Q12241709', 'wd:Q428602', 'wd:Q9826', 'wd:Q64063386', 'wd:Q149566', 'wd:Q875538', 'wd:Q159334', 'wd:Q3914', 'wd:Q2385804', 'wd:Q5341295', 'wd:Q902104', 'wd:Q1377182', 'wd:Q6540832', 'wd:Q494230', 'wd:Q1321960', 'wd:Q1143635', 'wd:Q20820271', 'wd:Q1663017', 'wd:Q423208', 'wd:Q11391922', 'wd:Q352102']

avg_len = 3
remainder = 0
sub_arrays = []
start = 0

for i in range(9):
    end = start + avg_len + (1 if i < remainder else 0)
    sub_arrays.append(categories_to_scan[start:end])
    start = end

categories = sub_arrays
print('running on categories:', categories)

In [4]:
def fetch_wikidata_entities(file):
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)
    
    for categories_subset in tqdm(categories, desc="Processing Categories"):
        query = f"""
            SELECT ?institution ?instance WHERE {{
            VALUES ?instance {{{' '.join(categories_subset)}}}
            {{
                ?institution wdt:P31 ?instance .
            }}
            UNION
            # Second layer: Subclasses of instances (subclass of a subclass)
            {{
                ?subclass wdt:P279* ?instance .
                ?institution wdt:P31* ?subclass .
            }}
            
            OPTIONAL {{ ?institution wdt:P17 ?country . }}
            FILTER (!BOUND(?country) || ?country = wd:Q30)
            }}

            GROUP BY ?institution ?instance
        """

        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        
        try:
            # Execute the query
            results = sparql.query().convert()['results']['bindings']
            result_lst.extend(results)
            for result in results:
                file.write(str(result))

            print(f'adding {len(results)} entries to result_lst')
            print(len(result_lst))
            # Process the results
        
        except Exception as e:
            print(f"An error occurred: {e}")
    return result_lst
    

In [5]:
def fetch_wikidata_entities_subset(file, categories_subset):
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)
    
    for category in tqdm(categories_subset, desc="Processing Categories"):
        query = f"""
            SELECT ?institution ?instance WHERE {{
            VALUES ?instance {{{category}}}

            ?institution wdt:P31 ?instance .
            
            OPTIONAL {{ ?institution wdt:P17 ?country . }}
            FILTER (!BOUND(?country) || ?country = wd:Q30)
            }}

            GROUP BY ?institution ?instance
        """

        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        
        try:
            # Execute the query
            results = sparql.query().convert()['results']['bindings']
            result_lst.extend(results)
            for result in results:
                file.write(str(result))

            print(f'adding {len(results)} entries to result_lst')
            print(len(result_lst))
            # Process the results
        
        except Exception as e:
            print(f"An error occurred: {e}")
        
        query = f"""
            SELECT ?institution ?instance WHERE {{
            VALUES ?instance {{{categories[4][1]}}}
            
            ?subclass wdt:P279 ?instance .
            ?institution wdt:P31 ?subclass .
            
            OPTIONAL {{ ?institution wdt:P17 ?country . }}
            FILTER (!BOUND(?country) || ?country = wd:Q30)
            }}

            GROUP BY ?institution ?instance
        """

        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        
        try:
            # Execute the query
            results = sparql.query().convert()['results']['bindings']
            result_lst.extend(results)
            for result in results:
                file.write(str(result))

            print(f'adding {len(results)} entries to result_lst')
            print(len(result_lst))
            # Process the results
        
        except Exception as e:
            print(f"An error occurred: {e}")
    return result_lst
    

In [43]:
file_name = f'wikidata_entites_{time.time()}.txt'

In [ ]:
with open(file_name, 'w') as file:
    df_entities = fetch_wikidata_entities(file)

In [ ]:
# Run on any categories that timed out
category = [categories[4][1]]

with open(file_name, 'w') as file:
    df_entities = fetch_wikidata_entities_subset(file, category)

In [45]:
df_entities = pd.DataFrame(df_entities)

In [ ]:
df_entities.shape

In [ ]:
df_entities.sample(5)

In [37]:
# remove duplicate ids
df_unique = df_entities.drop_duplicates(subset="institution")

In [ ]:
df_unique.shape

In [ ]:
df_unique.sample(5)

In [48]:
df_entities = df_unique

In [49]:
df_entities['institution_id'] = df_entities['institution'].apply(lambda x: x['value'].split('/')[-1])

# Write the extracted IDs to a text file
output_file = "institution_ids.txt"
with open(output_file, "w") as f:
    f.writelines(f"{inst_id}\n" for inst_id in df_entities['institution_id'])

# Get each entity

In [3]:
def process_entity(results: List[dict], entity_id):
    """
    Aggregates the input data into a dictionary where the key is valueLabel['value'] 
    and the value is a dictionary mapping propertyLabel['value'] to valueLabel['value'].

    Args:
        data (list of dict): List of dictionaries containing 'valueLabel' and 'propertyLabel'.

    Returns:
        dict: Aggregated dictionary.
    """
    aggregated = {}

    if results:
        for item in results:
            value_label = item["valueLabel"]["value"]
            property_label = item["propertyLabel"]["value"]

            # Initialize nested dictionary for the value_label if not already present
            if property_label not in aggregated:
                aggregated[property_label] = [value_label]
            else:
                aggregated[property_label].append(value_label)
        aggregated['wd_id'] = entity_id
        aggregated['name'] = results[0]['itemLabel']['value']
        return aggregated
    else:
        print("no results found")

In [4]:
def get_entity(match):
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)

    query = f"""
        SELECT ?itemLabel ?propertyLabel ?valueLabel ?altLabel WHERE {{
            BIND(wd:{match} AS ?item)  # Bind the input entity to ?item
            ?item ?prop ?value .
            ?property wikibase:directClaim ?prop .

            OPTIONAL {{
                ?value rdfs:label ?valueLabel .
                FILTER (lang(?valueLabel) = "en")
            }}

            OPTIONAL {{ ?item skos:altLabel ?altLabel . FILTER (LANG(?altLabel) = "en") }}

            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
        """
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    results = sparql.query().convert()
    results = results['results']['bindings']
    return results

# Execute

In [5]:
import time

def fetch_entity_with_retry(entity, max_retries=5, backoff_factor=1.5):
    retries = 0
    while retries < max_retries:
        try:
            # Replace this with the actual API call
            result = get_entity(entity)  # Your function that fetches the entity
            return result
        except SPARQLExceptions.QueryBadFormed as e:
            print("Bad query format:", e)
            break
        except Exception as e:
            # Check for Retry-After header in response
            if hasattr(e, "response") and e.response:
                retry_after = e.response.headers.get("Retry-After")
                if retry_after:
                    wait_time = int(retry_after)
                    print(f"Rate limit hit. Retrying after {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    print("Retry-After header not found. Retrying after 5 seconds...")
                    time.sleep(5)
            else:
                print(f"Unexpected error: {e}")
                break
        # except Exception as e:
        #     # If a 429 error occurs, handle it
        #         retries += 1
        #         wait_time = backoff_factor ** retries + random.uniform(0, 4)
        #         print(f"Rate limit hit. Retrying in {wait_time:.2f} seconds...")
        #         time.sleep(wait_time)
    print(f"Max retries reached. Skipping {entity}")
    return None  # Return None if retries are exhausted


In [ ]:
# ~ 1h20m

all_res = []
output_txt = f"raw_wiki_entities_{time.time()}.txt"
print(f'writing entities to {output_txt}')

with open('institution_ids_429_11_25.txt', 'r') as file:
    # Read all lines and store them in a list
    lines = file.readlines()

# Strip the newline characters from each line
results = [line.strip() for line in lines]
if results:
    for entity_id in tqdm(results, desc="Processing Entities", unit="entity", miniters=100, mininterval=0.5):
        entity_data = fetch_entity_with_retry(entity_id)  # Retry logic added here
        if entity_data:
            entity_data = process_entity(entity_data, entity_id)  # Process the entity after fetching
            with open(output_txt, mode="a", encoding="utf-8") as txtfile:
                # Format the entity data as a string for writing
                txtfile.write(f"{str(entity_data)}\n")
            
            all_res.append(entity_data)

In [ ]:
import ast
import json
import re


file_path = 'wiki_entities_11_25.txt'
data = []

# Open the file and read line by line
with open(file_path, 'r') as file:
    for i, line in enumerate(file):
        try:
            # Safely evaluate the Python-like dictionary format
            record = ast.literal_eval(line.strip())
            data.append(record)
        except json.JSONDecodeError as e:
            print(f"Error on line {i}: {line.strip()}")
            print(f"JSONDecodeError: {e}")

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

In [21]:
df.sample()

,coordinate location,Commons category,NLA Trove people ID,endowment,VIAF ID,Microsoft Academic ID,subreddit,HAL structure ID,Integrated Postsecondary Education Data System ID,LittleSis organization ID,...,intangible cultural heritage status,payload mass,theorised by,PubMed publication ID,author name string,Gateway to Research Project ID,principal investigator,funding scheme,Dictionary of Archives Terminology ID,officeholder
70212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
num_columns = df.shape[1]
print(num_columns)
df.columns

1510


Index(['coordinate location', 'Commons category', 'NLA Trove people ID',
       'endowment', 'VIAF ID', 'Microsoft Academic ID', 'subreddit',
       'HAL structure ID', 'Integrated Postsecondary Education Data System ID',
       'LittleSis organization ID',
       ...
       'intangible cultural heritage status', 'payload mass', 'theorised by',
       'PubMed publication ID', 'author name string',
       'Gateway to Research Project ID', 'principal investigator',
       'funding scheme', 'Dictionary of Archives Terminology ID',
       'officeholder'],
      dtype='object', length=1510)

# Get ids that were lost to 429 errors

In [ ]:
import ast
import json


institutions_file = 'institution_ids.txt'
output_file = "missing_ids.txt"

# Read institutions.txt
with open(institutions_file, "r") as f:
    institution_ids = {line.strip() for line in f.readlines()}

# Read processed_entities.txt and extract 'wd_id' values
processed_ids = set()
with open(processed_entities_file, "r") as f:
    for line in f:
        try:
            entity = ast.literal_eval(line.strip())
            
            # Check if 'wd_id' exists and process it
            if isinstance(entity, dict) and 'wd_id' in entity:
                processed_ids.add(entity['wd_id'])
        except json.JSONDecodeError:
            print(f"Error decoding line: {line.strip()}")  # Optional: handle or log malformed lines

# Compute the difference
missing_ids = institution_ids - processed_ids

# Write missing IDs to output file
with open(output_file, "w") as f:
    f.writelines(f"{wd_id}\n" for wd_id in missing_ids)

print(f"Missing IDs written to {output_file}")

In [ ]:
output_txt = f"processed_entities_{time.time()}.txt"
print(f'writing entities to {output_txt}')

# results = fetch_wikidata_entities()
results = missing_ids
if results:
    for entity_id in tqdm(results, desc="Processing Entities", unit="entity"):
        entity_data = fetch_entity_with_retry(entity_id)  # Retry logic added here
        if entity_data:
            entity_data = process_entity(entity_data, entity_id)  # Process the entity after fetching
            with open(output_txt, mode="a", encoding="utf-8") as txtfile:
                # Format the entity data as a string for writing
                txtfile.write(f"{str(entity_data)}\n")
            
            all_res.append(entity_data)

# Process data

In [ ]:
df = pd.DataFrame(all_res)

In [ ]:
df.sample()

In [ ]:
num_rows = df.shape[0]
print(num_rows)

In [23]:
print(df['name'].unique())

['Michigan Technological University' 'University of Massachusetts Amherst'
 'University of Massachusetts Boston' ... 'Q116134275' 'Q116134398'
 'Q116134482']


In [24]:
# Count rows before filtering
initial_count = len(df)

# Drop schools without names
df.drop(df[df['name'] == df['wd_id']].index, inplace=True)

# Count rows after filtering
final_count = len(df)

# Calculate and print the number of dropped rows
dropped_count = initial_count - final_count
print(f"Dropping {dropped_count} items without a name")

Dropping 16863 items without a name


In [25]:
# Count rows before filtering
initial_count = len(df)

# Drop items without instance of
df = df.dropna(subset=['instance of'])

# Count rows after filtering
final_count = len(df)

# Calculate and print the number of dropped rows
dropped_count = initial_count - final_count
print(f"Dropping {dropped_count} items without an instance of. {final_count} entities remaining.")

Dropping 926 items without an instance of. 58237 entities remaining.


In [26]:
# drop all columns where one or zero rows have value

initial_count = df.shape[1]

df = df.dropna(axis=1, how='all')
df = df.loc[:, df.notna().sum() > 1]
df = df[['name'] + ['wd_id'] + [col for col in df.columns if col != 'name' and col != 'wd_id']]

final_count = df.shape[1]
dropped_count = initial_count - final_count
print(f"Dropping {dropped_count} columns without any content")

Dropping 460 columns without any content


In [27]:
df.sample(5)

,name,wd_id,coordinate location,Commons category,NLA Trove people ID,endowment,VIAF ID,Microsoft Academic ID,subreddit,HAL structure ID,...,ICANNWiki page ID,discontinued date,editor,programmed in,IFTTT service ID,SlideShare username,WikiProjectMed ID,APA Dictionary of Psychology entry,NCI Thesaurus ID,J. Paul Getty Museum agent ID
14314,Edward M. Kennedy Academy for Health Careers,Q5344270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30007,Brighter Star Secondary School,Q4967521,[Point(87.686 26.6602)],NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8437,University of California San Francisco Departm...,Q101028858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49721,Woodridge School District 68,Q85816135,"[Point(-88.039738 41.7411379), Point(-88.03973...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64447,"Institute for Mind and Body, University of Chi...",Q112085273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
with open('columns.txt', 'w') as f:
    for column in df.columns:
        f.write(column + '\n')

# Write df to file

In [ ]:
file_name = f'processed_schools_{time.time()}.csv'

df.to_csv(file_name, index=False)

print(f'Wrote df to {file_name}')

In [ ]:
# Explode the 'instance of' column to turn lists into separate rows
df_exploded = df.explode('instance of')

# Count the occurrences of each value in 'instance of'
instance_counts = df_exploded['instance of'].value_counts()

# Display the counts
print(instance_counts)

In [ ]:
df_instance_counts = pd.DataFrame(instance_counts)

In [ ]:
with open('instance_counts.txt', 'w') as file:
    for value, count in instance_counts.items():
        file.write(f"{value}: {count}\n")

In [ ]:
# remove all rows with certain values for instanceof
instances_to_remove = ['faculty', 'organization', '']

In [ ]:
# Put items back into lists

In [ ]:
df = df_exploded.groupby('wd_id')['instance of'].agg(list).reset_index()

In [ ]:
df.sample(10)

In [ ]:
df['merged_column'] = df.apply(
    lambda row: row['located in the administrative territorial entity'] + row['located in the present-day administrative territorial entity'] + row['location'], axis=1
)

In [29]:
cols_to_keep = ['name', 'wd_id', 'country', 'instance of', 'parent organization', 'owner of', 'logo image', 'official website', 'Instagram username', 'X username', 'street address', 'admission rate', 'count of students', 'postal code', 'inception', 'has part(s)', 'image', 'Facebook username', 'has subsidiary', 'official name', 'headquarters location', 'LinkedIn company or organization ID', 'short name', 'location', 'YouTube channel ID', 'coat of arms image', 'seal image', 'owned by', 'board member', 'tuition fee', 'subclass of', 'IRS Employer Identification Number', 'nickname', 'operating area', 'small logo or icon']

subset_df = df[cols_to_keep]


In [30]:
subset_df.sample()

,name,wd_id,country,instance of,parent organization,owner of,logo image,official website,Instagram username,X username,...,coat of arms image,seal image,owned by,board member,tuition fee,subclass of,IRS Employer Identification Number,nickname,operating area,small logo or icon
22437,Midwood High School,Q10749664,[United States of America],[high school],NaN,NaN,NaN,[http://www.midwoodhighschool.org/],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
subset_df.shape

(58237, 35)

In [34]:
# drop all columns where one or zero rows have value

initial_count = subset_df.shape[1]

subset_df = subset_df.dropna(axis=1, how='all')
subset_df = subset_df.loc[:, subset_df.notna().sum() > 1]
subset_df = subset_df[['name'] + ['wd_id'] + ['nickname'] + [col for col in subset_df.columns if col != 'name' and col != 'wd_id' and col != 'nickname']]

final_count = subset_df.shape[1]
dropped_count = initial_count - final_count
print(f"Dropping {dropped_count} columns without any content")

Dropping 0 columns without any content


In [35]:
file_name = f'processed_schools_min_cols_{time.time()}.csv'

subset_df.to_csv(file_name, index=False)

print(f'Wrote df to {file_name}')

Wrote df to processed_schools_min_cols_1732548646.9518888.csv


In [64]:
# un-list items
import numpy as np

for col in subset_df.columns:
    subset_df[col] = subset_df[col].apply(
        lambda x: x[0] if isinstance(x, list) and len(x) == 1 else x
    )

In [ ]:
print(subset_df.head())
print(subset_df.dtypes)

In [37]:
subset_df.sample(5)

,name,wd_id,nickname,country,instance of,parent organization,owner of,logo image,official website,Instagram username,...,YouTube channel ID,coat of arms image,seal image,owned by,board member,tuition fee,subclass of,IRS Employer Identification Number,operating area,small logo or icon
12088,Phillips Academy,Q1432645,NaN,"[United States of America, United States of Am...","[high school, high school, high school, high s...",NaN,NaN,NaN,"[https://www.andover.edu/, https://www.andover...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53333,Beacon City Schools,Q4875911,NaN,[United States of America],[school district],NaN,NaN,NaN,[http://www.beaconcityk12.org/],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26931,Academy of Saint Joseph,Q4671524,NaN,[United States of America],[university-preparatory school],NaN,NaN,NaN,[http://www.asjli.org],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33560,Tennessee Virtual Academy,Q48977549,NaN,NaN,[school],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38555,Valencia College Lake Nona Campus,Q35190385,NaN,[United States of America],[academic library],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
subset_df['nickname'].value_counts()


nickname
[APU]                                                                                                                                                                                                                            1
[Brick City, Brick City, Brick City, Brick City, Brick City, Brick City, Brick City, Brick City, Brick City]                                                                                                                     1
[APUS, APUS]                                                                                                                                                                                                                     1
[Seminex, Seminex]                                                                                                                                                                                                               1
[Garden of Enid, Garden of Enid, Garden of Enid, Garden of Enid]                   

subset_df.col

# Merge in nicknames

In [8]:
file_name = 'processed_schools_min_cols_1732026738.218009.csv'

In [13]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
import random

def fetch_nicknames(wikidata_id):
    query = f"""
    SELECT ?itemLabel ?altLabel WHERE {{
        BIND(wd:{wikidata_id} AS ?item)  # Bind the input entity to ?item
        OPTIONAL {{ ?item skos:altLabel ?altLabel . FILTER (LANG(?altLabel) = "en") }}
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    
    # Execute the query and fetch results
    sparql.query()
    results = sparql.query().convert()

    # Extract nicknames (altLabels) from results
    nicknames = [result.get('altLabel')['value'] for result in results['results']['bindings'] if result.get('altLabel')]
    return nicknames

def fetch_entity_with_retry(entity, max_retries=5, backoff_factor=1.5):
    retries = 0
    while retries < max_retries:
        try:
            # Replace this with the actual API call
            result = fetch_nicknames(entity)  # Your function that fetches the entity
            return result
        except Exception as e:
            # If a 429 error occurs, handle it
            if isinstance(e, requests.exceptions.HTTPError) and e.response.status_code == 429:
                retries += 1
                wait_time = backoff_factor ** retries + random.uniform(0, 2)
                print(f"Rate limit hit. Retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
            else:
                retries += 1
                wait_time = backoff_factor ** retries + random.uniform(0, 2)
                print(f"Error getting {entity}, {e}")
                time.sleep(wait_time)
    print("Max retries reached. Skipping this entity.")
    return None  # Return None if retries are exhausted

def merge_nicknames_with_csv(csv_file, wikidata_column):
    # Read the CSV
    df = pd.read_csv(csv_file)
    
    # Create an empty list to store the nicknames
    nicknames_list = []

    # Iterate through each row and fetch the nicknames for the Wikidata ID
    for _index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Fetching Nicknames", unit="row"):
        wikidata_id = row[wikidata_column]
        nicknames = fetch_entity_with_retry(wikidata_id)
        nicknames_list.append(nicknames)

    # Add the nicknames as a new column to the DataFrame
    df['nicknames'] = nicknames_list
    
    # Save the updated DataFrame to a new CSV
    with tqdm(total=len(df), desc="Saving CSV", unit="row") as pbar:
        df.to_csv('updated_with_nicknames.csv', index=False)
        pbar.update(len(df))

In [ ]:
csv_file = file_name  # Replace with your existing CSV file path
wikidata_column = 'wd_id'  # Replace with the column containing Wikidata IDs (e.g., Q536709)

merge_nicknames_with_csv(csv_file, wikidata_column)

# Export as csv

In [ ]:
file_name = f'processed_schools_min_cols_{time.time()}.csv'

subset_df.to_csv(file_name, index=False)

print(f'Wrote df to {file_name}')

In [3]:
import pandas as pd

with open('schools.csv', 'r') as file:
    df = pd.read_csv(file)

In [ ]:
df.sample(5)